# Data Preparation

<hr>


In [1]:
# Imports
import ast
import json
import logging
import random
import re
import sqlite3
import time
import uuid
from datetime import date, datetime, timedelta
from sqlite3 import OperationalError

import numpy as np
import pandas as pd
import requests
from faker import Faker
from IPython.display import display
from passlib.context import CryptContext

In [2]:
# Connect to (or create) the SQLite database file
conn = sqlite3.connect("../data/vaccination_db.sqlite")
cursor = conn.cursor()

# Read the schema file and execute the SQL commands
with open("../data/schema.sql", "r") as f:
    sql_script = f.read()

try:
    cursor.executescript(sql_script)
except OperationalError:
    pass
finally:
    conn.commit()
    conn.close()

In [3]:
fake = Faker()
fake.seed_instance(42)
np.random.seed(42)

<hr>

### **OneMap API**

API Document Link: https://www.onemap.gov.sg/apidocs/search


In [4]:
def get_address_info(text_input: str) -> dict | None:
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={text_input}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    headers = {"Authorization": "Bearer **********************"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data["found"] > 0:  # If postal code exists in OneMap
            result = data["results"][0]
            return {
                "postal_code": result["POSTAL"],
                "address": result[
                    "ADDRESS"
                ],  # f"{(result['BLK_NO'] if result['BLK_NO'] != 'NIL' else '')} {(result['ROAD_NAME'] if result['ROAD_NAME'] != 'NIL' else '')} {(result['BUILDING'] if result['BUILDING'] != 'NIL' else '')}",
                "latitude": result["LATITUDE"],
                "longitude": result["LONGITUDE"],
            }
    return None  # Invalid postal code

<hr>

## Clinics Table

Note: The column `postal_code` will be later mapped to its corresponding `address_id`. It is stored first in order to populate the `Addresses` table.


### Polyclinics Data


In [5]:
# -----------------------------
# Load the JSON Polyclinic data
# -----------------------------
with open("../data/get_instituitions_response.json", "r") as f:
    institutions_data = json.load(f)

In [6]:
# -----------------------------
# Process JSON Polyclinic data
# -----------------------------
# Extract the list of health organizations
health_organizations = institutions_data["Result"]["HealthOrganizations"]

# Normalize the 'Cluster' and 'Group' columns
institutions_df = pd.json_normalize(health_organizations, sep="_")

# Sort the DataFrame by the 'Id' column
institutions_df = institutions_df.sort_values(by="Id").reset_index(drop=True)

# Fliter for polyclinics (Count: 26)
polyclinics_df = institutions_df[
    institutions_df["GroupName"] == "Polyclinics"
].reset_index()

# Data Check
print(f"Polyclinic Count: {len(polyclinics_df)}")
print(f"Unique Polyclinic Name Count: {len(polyclinics_df['Name'].unique())}")

# Extract relevant columns
include_columns = ["Name"]
polyclinics_df = polyclinics_df[include_columns]
# Rename to "name" column
polyclinics_df.rename(columns={"Name": "name"}, inplace=True)

# Get Postal Code using OneMap
polyclinics_df["postal_code"] = polyclinics_df["name"].apply(
    lambda name: (
        pd.Series(get_address_info(name), index=["postal_code"])
        if get_address_info(name)
        else pd.Series([None, None], index=["postal_code"])
    )
)

# Update incorrectly identified postal code(s)
update_postal_code = {"Sembawang Polyclinic": "756973"}
for polyclinic, postal_code in update_postal_code.items():
    polyclinics_df.loc[polyclinics_df["name"] == polyclinic, "postal_code"] = (
        postal_code
    )

# Add new column "type"
polyclinics_df["type"] = "polyclinic"

display(polyclinics_df)

Polyclinic Count: 26
Unique Polyclinic Name Count: 26


,name,postal_code,type
0,Ang Mo Kio Polyclinic,569666,polyclinic
1,Geylang Polyclinic,389707,polyclinic
2,Hougang Polyclinic,538829,polyclinic
3,Kallang Polyclinic,328263,polyclinic
4,Khatib Polyclinic,769567,polyclinic
5,Toa Payoh Polyclinic,319260,polyclinic
6,Sembawang Polyclinic,756973,polyclinic
7,Woodlands Polyclinic,738579,polyclinic
8,Yishun Polyclinic,768898,polyclinic
9,Bukit Batok Polyclinic,659164,polyclinic


### General Practitioners Data


In [7]:
# -------------------------------------
# Load clinic dataset from `data.gov.sg`
# -------------------------------------
# Define dataset ID and URL
dataset_id = "d_3cd840069e95b6a521aa5301a084b25a"
url = "https://data.gov.sg/api/action/datastore_search?resource_id=" + dataset_id

# Initialize an empty list to hold all records
all_clinic_records = []

# Set the initial offset and limit
limit = 100  # Max records per request
offset = 0

# Loop through the paginated data as the API only returns 100 records per request
while True:
    response = requests.get(f"{url}&limit={limit}&offset={offset}")

    if response.status_code == 200:
        data = response.json()
        records = data["result"]["records"]

        all_clinic_records.extend(records)

        # Check if there are more records (if the number of records returned is less than the limit, stop)
        if len(records) < limit:
            break

        # Update offset for the next batch of records
        offset += limit
    else:
        print(f"Request failed with status code {response.status_code}")
        break

data_gov_clinic_df = pd.DataFrame(all_clinic_records)

In [8]:
# ------------
# Process data
# ------------
# Get General Practitioners only
gp_api_df = data_gov_clinic_df[
    data_gov_clinic_df["category"].isin(["gp", "non-chas-gp"])
]
print(f"GP Count: {len(gp_api_df)}")
# There may be different outlets across Singapore
print(f"Unique GP Name Count: {len(gp_api_df['name'].unique())}")
print(f"Unique GP Postal Code Count: {len(gp_api_df['postal_code'].unique())}")

gps_df = gp_api_df.copy()
# Extract the required columns
columns_to_extract = ["name", "postal_code"]
gps_df = gps_df[columns_to_extract].reset_index(drop=True)

# Add new column "type"
gps_df["type"] = "gp"

display(gps_df)

GP Count: 1532
Unique GP Name Count: 1486
Unique GP Postal Code Count: 1074


,name,postal_code,type
0,Cavenagh Medical Clinic And Home Care,269695,gp
1,Mei Ling Clinic,140158,gp
2,Rcmc Rivervale Crescent Medical Centre,541182,gp
3,360 Clinic,560407,gp
4,SKY Medical,079027,gp
...,...,...,...
1527,Trinity Medical@Shenton,068908,gp
1528,Saudara Clinic By A+J General Physicians,419741,gp
1529,Zara Clinic,680026,gp
1530,360 Clinic,640221,gp


### Combined Clinics Data


In [9]:
# Clinics Table
clinics_data = {
    "id": [
        str(uuid.uuid4()) for _ in range(1, len(gps_df) + len(polyclinics_df) + 1)
    ],  # Generate a UUID v4 as a string
    "name": polyclinics_df["name"].tolist() + gps_df["name"].tolist(),
    "postal_code": polyclinics_df["postal_code"].tolist()
    + gps_df["postal_code"].tolist(),
    "type": polyclinics_df["type"].tolist() + gps_df["type"].tolist(),
}
clinics_df = pd.DataFrame(clinics_data)

# Get list of unique gp postal_code
clinic_postal_code = clinics_df["postal_code"].drop_duplicates().tolist()
print(f"Clinics `postal_code` Unique Count: {len(clinic_postal_code)}")
display(clinics_df)

Clinics `postal_code` Unique Count: 1096


,id,name,postal_code,type
0,ca848978-29e9-4345-8233-6d968543cbec,Ang Mo Kio Polyclinic,569666,polyclinic
1,5244d263-c742-415b-aa42-9c79e745d272,Geylang Polyclinic,389707,polyclinic
2,3010c7a1-52b2-4df1-9922-8c67236ca4e3,Hougang Polyclinic,538829,polyclinic
3,00ce402a-08ce-4be5-85d8-ad176779c60c,Kallang Polyclinic,328263,polyclinic
4,d66f950a-694e-435c-b0af-ea43f80391a0,Khatib Polyclinic,769567,polyclinic
...,...,...,...,...
1553,918370f4-fbe8-44c0-9017-7492fc922661,Trinity Medical@Shenton,068908,gp
1554,98ab4ef0-9344-4f8a-acf1-663e193f03d8,Saudara Clinic By A+J General Physicians,419741,gp
1555,20055a21-31ec-46d0-8a3a-c562089bf9cd,Zara Clinic,680026,gp
1556,f4f9e8f6-1416-4b21-993b-86bee5c62684,360 Clinic,640221,gp


<hr>

## Users Table


### Functions


In [10]:
# Regex pattern for nric
nric_regex = r"[STFGM]\d{7}[A-Z]"


def generate_nric(dob: date) -> str:
    year = dob.year
    year_suffix = str(year % 100).zfill(2)  # Extract last two digits of the year
    serial_number = str(random.randint(10000, 99999))  # Ensure exactly 7 digits
    checksum = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZ")  # Random letter for checksum

    # Determine prefix based on birth year
    prefix = "S" if year < 2000 else "T"

    sample_nric = f"{prefix}{year_suffix}{serial_number}{checksum}"

    # Validate against regex before returning
    if not re.fullmatch(nric_regex, sample_nric):
        raise ValueError(
            f"Generated NRIC '{sample_nric}' does not match expected pattern."
        )

    return sample_nric


def generate_email(first_name, last_name):
    email_username = f"{first_name.lower()}.{last_name.lower()}"
    email_domain = fake.free_email_domain()  # Use a real email domain
    return f"{email_username}@{email_domain}"


# Set up password hashing
logging.getLogger("passlib").setLevel(logging.ERROR)
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")


def hash_password(password: str) -> str:
    return pwd_context.hash(password)

### Generate Users Data


In [11]:
N = 5

ids = []
enrolled_clinic_ids = []
nrics = []
first_names = []
last_names = []
emails = []
date_of_births = []
genders = []
passwords = ["password123", "abc123", "123", "admin", "guest"]

polyclinic_id_list = clinics_df[clinics_df["type"] == "polyclinic"]["id"].tolist()
gp_id_list = clinics_df[clinics_df["type"] == "gp"]["id"].tolist()

for _ in range(N):
    # Generate data
    gender = np.random.choice(["M", "F"], p=[0.5, 0.5])
    first_name = fake.first_name_male() if gender == "M" else fake.first_name_female()
    last_name = fake.last_name_male() if gender == "M" else fake.last_name_female()
    email = (
        first_name.lower() + "." + last_name.lower() + "@" + fake.email().split("@")[1]
    )
    date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=80)
    nric = generate_nric(date_of_birth)

    if np.random.rand() < 0.7:  # 70% chance for polyclinics
        enrolled_clinic_id = np.random.choice(polyclinic_id_list)
    elif np.random.rand() < 0.2:  # 20% chance for GPs
        enrolled_clinic_id = np.random.choice(gp_id_list)
    else:  # 10% chance for None
        enrolled_clinic_id = None

    # Append to list
    ids.append(str(uuid.uuid4()))
    nrics.append(nric)
    first_names.append(first_name)
    last_names.append(last_name)
    emails.append(email)
    date_of_births.append(date_of_birth)
    genders.append(gender)
    enrolled_clinic_ids.append(enrolled_clinic_id)

# Users Table
users_data = {
    "id": ids,
    "enrolled_clinic_id": enrolled_clinic_ids,
    "nric": nrics,
    "first_name": first_names,
    "last_name": last_names,
    "email": emails,
    "date_of_birth": date_of_births,
    "gender": genders,
    "password": [hash_password(password) for password in passwords],
}

users_df = pd.DataFrame(users_data)
display(users_df)

,id,enrolled_clinic_id,nric,first_name,last_name,email,date_of_birth,gender,password
0,812cf9f3-05be-429b-84d8-60b0fccac402,None,S9489143A,Mark,Johnson,mark.johnson@example.net,1994-08-27,M,$2b$12$kly/wBEGaHIDWw.1RWGNtOoz7I1UZbOqw7ShG8s...
1,b14a41d8-5819-4685-93fa-d46434f32e0a,042e6bc3-6cd0-4808-909c-16fbde02d4ee,S6091645N,Stephanie,Miller,stephanie.miller@example.org,1960-01-31,F,$2b$12$Zpo8JyOjvPF2sHBZzakUO.WYt8wIsT9pp.2qXNL...
2,d9d303b7-240c-4320-ae6c-e3d7ef914ce3,a0e4b2dc-4bff-414f-ae07-00584aef455a,S5962037V,Jonathan,Johnson,jonathan.johnson@example.com,1959-04-03,M,$2b$12$LGg25.ziP0jnuamBfeRx6.UvzRHcqLnyZjajdcB...
3,fcef2a1c-a9bc-4c27-a7bd-3aedc4ce4c53,4acb36b8-ffba-4db9-9893-70a3a81c8349,S6789723Q,Joann,Ramirez,joann.ramirez@example.com,1967-05-28,F,$2b$12$Md3jkP7vNprR2H/FZQOOeuG6X/9j6qdt.zAKEkS...
4,4d1ae0f6-dcbe-4b30-ba7d-9df9e1e9a95a,dd30df9a-de3b-401c-890c-66b33d556bbd,T0144555I,Diana,Lewis,diana.lewis@example.com,2001-12-03,F,$2b$12$GY7xUYyMlGQlVQ1EzWsNqujEMvp2UGNXllBRFrN...


### Generate `postal_code` for `Users` table

The column `postal_code` will be later mapped to its corresponding `address_id`. It is generated first in order to populate the `Addresses` table.

To generate the `postal_code` for `Users`, run either of the two cells:


In [12]:
valid_postal_codes = []
checked_codes = set()
postal_code_filepath = "../data/users_postal_codes.txt"

try:
    # (1) Try to load postal codes from .txt file
    with open(postal_code_filepath, "r") as file:
        valid_postal_codes = [line.strip() for line in file.readlines()]

    # Convert to actual dictionary
    valid_postal_codes = [ast.literal_eval(entry) for entry in valid_postal_codes]
    print("Loaded users' postal codes from txt file")

except FileNotFoundError:
    print(f"{postal_code_filepath} not found. Generating postal codes...")

    # (2) Generate postal codes if .txt file is not found
    while len(valid_postal_codes) < N:
        random_code = str(
            random.randint(100000, 999999)
        )  # Generate random 6-digit postal code

        if random_code in clinic_postal_code or random_code in checked_codes:
            continue  # Skip if already checked or is a clinic postal code

        checked_codes.add(random_code)
        geoData = get_address_info(random_code)

        if geoData:
            valid_postal_codes.append(geoData)
            print(
                f"Valid: {random_code} (Lat: {geoData['latitude']}, Lon: {geoData['longitude']}), Address: {geoData['address']}"
            )

        time.sleep(0.5)  # Sleep to prevent hitting API rate limits

    # Write postal codes to a .txt file for faster setup next time
    with open(postal_code_filepath, "w") as file:
        for code in valid_postal_codes:
            file.write(f"{code}\n")

Loaded users' postal codes from txt file


In [13]:
# List of generated postal codes
users_postal_codes = [entry["postal_code"] for entry in valid_postal_codes][:N]
users_df["postal_code"] = users_postal_codes

display(users_df)

,id,enrolled_clinic_id,nric,first_name,last_name,email,date_of_birth,gender,password,postal_code
0,812cf9f3-05be-429b-84d8-60b0fccac402,None,S9489143A,Mark,Johnson,mark.johnson@example.net,1994-08-27,M,$2b$12$kly/wBEGaHIDWw.1RWGNtOoz7I1UZbOqw7ShG8s...,249393
1,b14a41d8-5819-4685-93fa-d46434f32e0a,042e6bc3-6cd0-4808-909c-16fbde02d4ee,S6091645N,Stephanie,Miller,stephanie.miller@example.org,1960-01-31,F,$2b$12$Zpo8JyOjvPF2sHBZzakUO.WYt8wIsT9pp.2qXNL...,258201
2,d9d303b7-240c-4320-ae6c-e3d7ef914ce3,a0e4b2dc-4bff-414f-ae07-00584aef455a,S5962037V,Jonathan,Johnson,jonathan.johnson@example.com,1959-04-03,M,$2b$12$LGg25.ziP0jnuamBfeRx6.UvzRHcqLnyZjajdcB...,208511
3,fcef2a1c-a9bc-4c27-a7bd-3aedc4ce4c53,4acb36b8-ffba-4db9-9893-70a3a81c8349,S6789723Q,Joann,Ramirez,joann.ramirez@example.com,1967-05-28,F,$2b$12$Md3jkP7vNprR2H/FZQOOeuG6X/9j6qdt.zAKEkS...,109923
4,4d1ae0f6-dcbe-4b30-ba7d-9df9e1e9a95a,dd30df9a-de3b-401c-890c-66b33d556bbd,T0144555I,Diana,Lewis,diana.lewis@example.com,2001-12-03,F,$2b$12$GY7xUYyMlGQlVQ1EzWsNqujEMvp2UGNXllBRFrN...,618490


<hr>

## Addresses Table


In [14]:
# List all unique postal codes
# 5 (users) + 26 (polyclinics) + 1074 (gps) = 1105 postal_codes (expected) - 4 (duplicates) = 1101 postal_codes
all_postal_code = list(dict.fromkeys(clinic_postal_code + users_postal_codes))

address_data_filepath = "../data/addresses_data.json"
try:
    # (1) Try to load address data from JSON file
    with open(address_data_filepath, "r") as file:
        addresses_data = [json.loads(line) for line in file]
    print("Loaded addresses_data from JSON file")
except FileNotFoundError:
    print(f"{address_data_filepath} not found. Generating addresses_data...")
    # (2) Get the geo data from OneMap API, runtime: ~2mins
    addresses_data = [get_address_info(postal_code) for postal_code in all_postal_code]
    # Save data as JSON lines to preserve the structure
    with open(address_data_filepath, "w") as file:
        for entry in addresses_data:
            file.write(json.dumps(entry) + "\n")  # Each line is a JSON object

addresses_df = pd.DataFrame(addresses_data)

# Create 'id` column`
addresses_df["id"] = [str(uuid.uuid4()) for _ in range(1, len(addresses_df) + 1)]
# Insert 'id' as the first column
addresses_df.insert(0, "id", addresses_df.pop("id"))

display(addresses_df)

Loaded addresses_data from JSON file


,id,postal_code,address,latitude,longitude
0,c83ac8b7-8506-4408-95ae-8cc0e5079c2a,569666,21 ANG MO KIO CENTRAL 2 ANG MO KIO POLYCLINIC ...,1.3743245905856,103.845677779279
1,759cbf94-0cd4-4e14-844c-20bb6f3929c5,389707,21 GEYLANG EAST CENTRAL GEYLANG POLYCLINIC SIN...,1.31949365581957,103.887166041622
2,1c364832-89e7-453b-b846-88841cc26727,538829,89 HOUGANG AVENUE 4 HOUGANG POLYCLINIC SINGAPO...,1.3699068381066,103.88900146446
3,a97e81bd-f4c2-4f34-a8cd-a34f40fb8c89,328263,701 SERANGOON ROAD KALLANG POLYCLINIC AND LONG...,1.31678496747374,103.858752270079
4,962c3c1c-8809-4182-a918-4c0e14682f32,769567,690 YISHUN RING ROAD KHATIB POLYCLINIC SINGAPO...,1.41820402220914,103.834428037791
...,...,...,...,...,...
1096,eb3c6dee-c95e-4595-92f5-9b574702369d,249393,6 HOOT KIAM ROAD SINGAPORE 249393,1.29616017810173,103.830433349404
1097,6708a9a1-88fe-47f2-80a3-518a01d75fe9,258201,22 ROBIN ROAD SINGAPORE 258201,1.31757040476181,103.828200554188
1098,059d2e2a-8b8d-466c-a644-19ab8039288b,208511,101 KITCHENER ROAD JALAN BESAR PLAZA SINGAPORE...,1.30865766748104,103.858066663176
1099,9c1728ef-581c-4956-a8ab-ee77eb775be8,109923,3 BUKIT CHERMIN ROAD LEARNING SEEDS CHILD DEVE...,1.26882604863741,103.811916195226


<hr>

## Replace `postal_code` column with `address_id` in `Clinics` and `Users` tables


In [15]:
# Mapping of postal_codes to address_ids from addresses_df
postal_to_address_id_mapping = dict(
    zip(addresses_df["postal_code"], addresses_df["id"])
)

# Map address_id to postal_code
clinics_df["address_id"] = clinics_df["postal_code"].map(postal_to_address_id_mapping)
users_df["address_id"] = users_df["postal_code"].map(postal_to_address_id_mapping)

# Select columns
clinics_df = clinics_df[["id", "address_id", "name", "type"]]
users_df = users_df[
    [
        "id",
        "address_id",
        "enrolled_clinic_id",
        "nric",
        "first_name",
        "last_name",
        "email",
        "date_of_birth",
        "gender",
        "password",
    ]
]

display(clinics_df)
display(users_df)

,id,address_id,name,type
0,ca848978-29e9-4345-8233-6d968543cbec,c83ac8b7-8506-4408-95ae-8cc0e5079c2a,Ang Mo Kio Polyclinic,polyclinic
1,5244d263-c742-415b-aa42-9c79e745d272,759cbf94-0cd4-4e14-844c-20bb6f3929c5,Geylang Polyclinic,polyclinic
2,3010c7a1-52b2-4df1-9922-8c67236ca4e3,1c364832-89e7-453b-b846-88841cc26727,Hougang Polyclinic,polyclinic
3,00ce402a-08ce-4be5-85d8-ad176779c60c,a97e81bd-f4c2-4f34-a8cd-a34f40fb8c89,Kallang Polyclinic,polyclinic
4,d66f950a-694e-435c-b0af-ea43f80391a0,962c3c1c-8809-4182-a918-4c0e14682f32,Khatib Polyclinic,polyclinic
...,...,...,...,...
1553,918370f4-fbe8-44c0-9017-7492fc922661,caa88de6-c9e2-4dfd-bb37-8bb21b56bfce,Trinity Medical@Shenton,gp
1554,98ab4ef0-9344-4f8a-acf1-663e193f03d8,4256935f-fb2b-4d1f-8ea4-78bfc1a4df53,Saudara Clinic By A+J General Physicians,gp
1555,20055a21-31ec-46d0-8a3a-c562089bf9cd,4a1d1d1b-765b-428b-8733-eba7bf8bbe76,Zara Clinic,gp
1556,f4f9e8f6-1416-4b21-993b-86bee5c62684,79e2aa5a-6dac-4b23-93b4-6693965b87e9,360 Clinic,gp


,id,address_id,enrolled_clinic_id,nric,first_name,last_name,email,date_of_birth,gender,password
0,812cf9f3-05be-429b-84d8-60b0fccac402,eb3c6dee-c95e-4595-92f5-9b574702369d,None,S9489143A,Mark,Johnson,mark.johnson@example.net,1994-08-27,M,$2b$12$kly/wBEGaHIDWw.1RWGNtOoz7I1UZbOqw7ShG8s...
1,b14a41d8-5819-4685-93fa-d46434f32e0a,6708a9a1-88fe-47f2-80a3-518a01d75fe9,042e6bc3-6cd0-4808-909c-16fbde02d4ee,S6091645N,Stephanie,Miller,stephanie.miller@example.org,1960-01-31,F,$2b$12$Zpo8JyOjvPF2sHBZzakUO.WYt8wIsT9pp.2qXNL...
2,d9d303b7-240c-4320-ae6c-e3d7ef914ce3,059d2e2a-8b8d-466c-a644-19ab8039288b,a0e4b2dc-4bff-414f-ae07-00584aef455a,S5962037V,Jonathan,Johnson,jonathan.johnson@example.com,1959-04-03,M,$2b$12$LGg25.ziP0jnuamBfeRx6.UvzRHcqLnyZjajdcB...
3,fcef2a1c-a9bc-4c27-a7bd-3aedc4ce4c53,9c1728ef-581c-4956-a8ab-ee77eb775be8,4acb36b8-ffba-4db9-9893-70a3a81c8349,S6789723Q,Joann,Ramirez,joann.ramirez@example.com,1967-05-28,F,$2b$12$Md3jkP7vNprR2H/FZQOOeuG6X/9j6qdt.zAKEkS...
4,4d1ae0f6-dcbe-4b30-ba7d-9df9e1e9a95a,fe68a41d-6ee2-449b-9b5a-68103a7f676f,dd30df9a-de3b-401c-890c-66b33d556bbd,T0144555I,Diana,Lewis,diana.lewis@example.com,2001-12-03,F,$2b$12$GY7xUYyMlGQlVQ1EzWsNqujEMvp2UGNXllBRFrN...


<hr>

## Vaccines & Vaccine Criteria Tables

In Singapore, there is the National Childhood Immnisation Schedule (NCIS) and National Adult Immunisation Schedule (NAIS).

Reference: https://www.moh.gov.sg/seeking-healthcare/overview-of-diseases/communicable-diseases/nationally-recommended-vaccines


### Process NAIS and NCIS Dataframes

In [16]:
# Load the Excel files
adult_vaccine_file = r"..\data\Singapore_National_Adult_Immunisation_Schedule.xlsx"
child_vaccine_file = r"..\data\Singapore_National_Childhood_Immunisation_Schedule.xlsx"

df_adult = pd.read_excel(adult_vaccine_file)
df_child = pd.read_excel(child_vaccine_file)

In [17]:
def expand_doses(df: pd.DataFrame) -> pd.DataFrame:
    """Get all permutations of vaccination recommendations into their own rows."""
    df = df.copy()

    # Replace NaN with empty strings to avoid errors
    df["Age Criteria"] = df["Age Criteria"].fillna("").astype(str).str.split(", ")
    df["Health Conditions"] = (
        df["Health Conditions"].fillna("").astype(str).str.split(", ")
    )
    df["Dose Required"] = df["Dose Required"].fillna("").astype(str).str.split(", ")

    # Ensure each row's lists have the same length
    max_len = df.apply(
        lambda row: max(
            len(row["Age Criteria"]),
            len(row["Health Conditions"]),
            len(row["Dose Required"]),
        ),
        axis=1,
    )

    def pad_list(lst, length):
        """Pads shorter lists with the last valid value to ensure alignment."""
        if len(lst) == 1 and length > 1:
            lst = lst * length  # Duplicate single value
        return lst + [lst[-1]] * (length - len(lst))  # Fill with last value

    df["Age Criteria"] = df.apply(
        lambda row: pad_list(row["Age Criteria"], max_len[row.name]), axis=1
    )
    df["Health Conditions"] = df.apply(
        lambda row: pad_list(row["Health Conditions"], max_len[row.name]), axis=1
    )
    df["Dose Required"] = df.apply(
        lambda row: pad_list(row["Dose Required"], max_len[row.name]), axis=1
    )

    # Explode all three columns together
    df = df.explode(["Age Criteria", "Health Conditions", "Dose Required"]).reset_index(
        drop=True
    )

    # Replace empty strings or NaN with None to maintain consistency
    df.replace("", "None", inplace=True)
    df.replace(np.nan, "None", inplace=True)

    return df

In [18]:
# Process adult vaccine schedule
df_adult_cleaned = expand_doses(df_adult)
df_adult_cleaned.rename(
    columns={
        "Age Criteria": "Age Group",
        "Gender Criteria": "Gender",
        "Vaccine Name": "Vaccine Name",
        "Dose Required": "Doses Required",
    },
    inplace=True,
)

# Process child vaccine schedule
df_child_cleaned = expand_doses(df_child)
df_child_cleaned.rename(
    columns={
        "Age Criteria": "Age Group",
        "Gender Criteria": "Gender",
        "Vaccine Name": "Vaccine Name",
        "Dose Required": "Doses Required",
    },
    inplace=True,
)
# Replace "Birth" with "0 months"
df_child_cleaned["Age Group"] = df_child_cleaned["Age Group"].replace(
    "Birth", "0 months"
)

# Combine NAIS and NCIS
df_vaccine_reco = pd.concat([df_adult_cleaned, df_child_cleaned], ignore_index=True)

In [19]:
# Create "Frequency" column
df_vaccine_reco["Frequency"] = df_vaccine_reco["Doses Required"].apply(
    lambda x: (
        "Annually or per season"
        if "annually" in x.lower()
        else "Per pregnancy" if "pregnancy" in x.lower() else "Once"
    )
)

# Get list of single dose from "Doses Required" column
single_dose_list = [
    "Dose 1",
    "Dose 2",
    "Dose 3",
    "1 dose annually or per season",
    "Booster 1",
    "Booster 2",
    "1 dose",
    "1 dose during each pregnancy",
]


def extract_dose_count(value):
    if value in single_dose_list:
        return 1
    # Extract first numeric value
    match = re.search(r"\b(\d+)\b", value)
    return int(match.group(1)) if match else None


# Update "Doses Required" column to reflect number of doses
df_vaccine_reco["Doses Required"] = df_vaccine_reco["Doses Required"].apply(
    extract_dose_count
)
# Format "Gender" column values
df_vaccine_reco["Gender"] = df_vaccine_reco["Gender"].apply(
    lambda x: "F" if x == "Female" else "M" if x == "Male" else x
)
df_vaccine_reco

,Vaccination Name,Age Group,Health Conditions,Doses Required,Gender,Frequency
0,Influenza (INF),18-64 years,Specific medical conditions or indications,1,None,Annually or per season
1,Influenza (INF),65+ years,None,1,None,Annually or per season
2,Pneumococcal conjugate (PCV13),18-64 years,Specific medical conditions or indications,1,None,Once
3,Pneumococcal conjugate (PCV13),65+ years,None,1,None,Once
4,Pneumococcal polysaccharide (PPSV23),18-64 years,Depending on indication,1,None,Once
5,Pneumococcal polysaccharide (PPSV23),65+ years,None,1,None,Once
6,"Tetanus, reduced diphtheria and acellular pert...",18+ years,Pregnancy,1,None,Per pregnancy
7,Human papillomavirus (HPV2 or HPV4),18-26 years,Unvaccinated adults or uncertain history,3,F,Once
8,Hepatitis B (HepB),18+ years,Unvaccinated adults or uncertain history,3,None,Once
9,"Measles, mumps and rubella (MMR)",18+ years,Unvaccinated adults or uncertain history,2,None,Once


### Vaccines Table

In [20]:
# Get list of vaccines
vaccines_list = df_vaccine_reco["Vaccination Name"].unique().tolist()

# Vaccines Table
vaccines_data = {
    "id": [str(uuid.uuid4()) for _ in range(0, len(vaccines_list))],
    "name": [vaccine for vaccine in vaccines_list],
}
vaccines_df = pd.DataFrame(vaccines_data)
display(vaccines_df)

,id,name
0,b79e4769-a79f-4142-96fc-da07d538cf9e,Influenza (INF)
1,681b0970-aac4-4e10-b71a-95904751de8a,Pneumococcal conjugate (PCV13)
2,cb5b468c-5403-465e-8980-3105fc0a6570,Pneumococcal polysaccharide (PPSV23)
3,faebabbd-8d6d-4f2f-b54c-5dfbe411ae11,"Tetanus, reduced diphtheria and acellular pert..."
4,b6d6ead4-ab98-4d83-9057-c0f1a7347d6f,Human papillomavirus (HPV2 or HPV4)
5,82104f64-fa90-41b2-a48b-4418cfebcc4b,Hepatitis B (HepB)
6,e9b88fcc-5474-44c5-b040-a5b3c21571ce,"Measles, mumps and rubella (MMR)"
7,6561d897-3ca3-44da-bb66-551d5ec104b2,Varicella (VAR)
8,b5e16dea-3fec-4449-a494-61ca2c85a0db,Bacillus Calmette-Guérin (BCG)
9,94d10a24-04eb-4c18-bb42-ad6d7132efe2,"Diphtheria, tetanus and acellular pertussis (D..."


### VaccineCriteria

In [21]:
# Vaccine Criteria Table
vaccine_criteria_data = {
    "id": [str(uuid.uuid4()) for _ in range(0, len(df_vaccine_reco))],
    "vaccine_id": df_vaccine_reco["Vaccination Name"].map(
        dict(zip(vaccines_df["name"], vaccines_df["id"]))
    ),
    "age_criteria": df_vaccine_reco["Age Group"].tolist(),
    "gender_criteria": df_vaccine_reco["Gender"].tolist(),
    "health_condition_criteria": df_vaccine_reco["Health Conditions"].tolist(),
    "doses_required": df_vaccine_reco["Doses Required"].tolist(),
    "frequency": df_vaccine_reco["Frequency"].tolist(),
}
vaccine_criteria_df = pd.DataFrame(vaccine_criteria_data)
display(vaccine_criteria_df)

,id,vaccine_id,age_criteria,gender_criteria,health_condition_criteria,doses_required,frequency
0,126d0049-383f-4978-9c4c-a89c23728687,b79e4769-a79f-4142-96fc-da07d538cf9e,18-64 years,None,Specific medical conditions or indications,1,Annually or per season
1,617c7996-546a-4b6c-b5bb-3e2cdd3bf0a1,b79e4769-a79f-4142-96fc-da07d538cf9e,65+ years,None,None,1,Annually or per season
2,8bdca1b3-dceb-4fd4-b8dd-d5398fc42133,681b0970-aac4-4e10-b71a-95904751de8a,18-64 years,None,Specific medical conditions or indications,1,Once
3,34ae6224-b415-4989-956b-050865b08be3,681b0970-aac4-4e10-b71a-95904751de8a,65+ years,None,None,1,Once
4,a8bf08bd-2d54-4da4-a92b-6536139ac3f3,cb5b468c-5403-465e-8980-3105fc0a6570,18-64 years,None,Depending on indication,1,Once
5,15d78fae-31fd-4680-8fe0-f9e7bb3d3b20,cb5b468c-5403-465e-8980-3105fc0a6570,65+ years,None,None,1,Once
6,1e4f3f91-8dbb-4c63-9174-7f05167ece9e,faebabbd-8d6d-4f2f-b54c-5dfbe411ae11,18+ years,None,Pregnancy,1,Per pregnancy
7,ba6d8ab5-4532-462e-95f4-18dde61a1765,b6d6ead4-ab98-4d83-9057-c0f1a7347d6f,18-26 years,F,Unvaccinated adults or uncertain history,3,Once
8,cd79fd4e-ffa4-48b4-b84d-cdc5814da685,82104f64-fa90-41b2-a48b-4418cfebcc4b,18+ years,None,Unvaccinated adults or uncertain history,3,Once
9,2c02d8df-9b3a-4e8f-965d-95ad8f4dbab3,e9b88fcc-5474-44c5-b040-a5b3c21571ce,18+ years,None,Unvaccinated adults or uncertain history,2,Once


<hr>

## BookingSlots Table


In [22]:
# Define start and end dates for March and April 2025
start_date = datetime(2025, 3, 1)
end_date = datetime(2025, 4, 30)

# Calculate number of booking slots: weekdays only (Monday to Friday)
weekday_slots_per_day = 18  # 8 AM to 5 PM in 30-min intervals, 18 slots per day
N = 0  # Track number of generated slots

ids = []
polyclinic_ids = []
vaccine_ids = []
datetimes = []

# BookingSlots will only be for Polyclinics
polyclinic_id_list = clinics_df[clinics_df["type"] == "polyclinic"]["id"].tolist()

# Loop through each day in March and April 2025
for day in range((end_date - start_date).days + 1):
    current_day = start_date + timedelta(days=day)

    # Check if the current day is a weekday (0 to 4 are weekdays)
    if current_day.weekday() < 5:  # 0 = Monday, 4 = Friday
        # Randomly select half of the polyclinics for this day
        selected_polyclinics = random.sample(
            polyclinic_id_list, len(polyclinic_id_list) // 2
        )

        for polyclinic_id in selected_polyclinics:  # 13 polyclinics
            for hour in range(8, 18):  # 8 AM to 5 PM
                for minute in [0, 30]:  # 0 for AM slots, 30 for PM slots
                    slot_time = current_day.replace(
                        hour=hour, minute=minute, second=0, microsecond=0
                    )

                    vaccine_id = random.choice(vaccines_df["id"].tolist())

                    # Append data for this slot
                    ids.append(str(uuid.uuid4()))
                    polyclinic_ids.append(polyclinic_id)
                    vaccine_ids.append(vaccine_id)
                    datetimes.append(slot_time)
                    N += 1

# Booking Slots Table
booking_slots_data = {
    "id": ids,
    "polyclinic_id": polyclinic_ids,
    "vaccine_id": vaccine_ids,
    "datetime": datetimes,
}
booking_slots_df = pd.DataFrame(booking_slots_data)
# Sort DataFrame by the 'datetime' column
booking_slots_df = booking_slots_df.sort_values(by="datetime").reset_index(drop=True)

display(booking_slots_df)

,id,polyclinic_id,vaccine_id,datetime
0,ba19c4e2-88a1-4feb-881b-38f0aa0da73c,dbcd1f73-cb22-484f-ab60-a8b931bd6033,cb5b468c-5403-465e-8980-3105fc0a6570,2025-03-03 08:00:00
1,161c078b-fb75-40b8-a850-311657fffc0a,5ac611d4-abfe-4008-a254-508ea1038161,82104f64-fa90-41b2-a48b-4418cfebcc4b,2025-03-03 08:00:00
2,b27d7807-69a5-4408-93ea-c08b54895001,adc2e872-c0bf-499a-bf0f-3f6bc057e5fb,681b0970-aac4-4e10-b71a-95904751de8a,2025-03-03 08:00:00
3,63e49af2-503c-4ab0-9de9-16df11f98671,7a6bc0a2-c61c-4129-8740-7dd7e5cc5bef,faebabbd-8d6d-4f2f-b54c-5dfbe411ae11,2025-03-03 08:00:00
4,db921906-c271-4f46-b0b0-476f171ab14d,db8e14db-f153-4f84-ab0d-439d3dfb1a0a,b6d6ead4-ab98-4d83-9057-c0f1a7347d6f,2025-03-03 08:00:00
...,...,...,...,...
11175,1d6f4133-f222-4442-a00f-7091198513cc,a0e4b2dc-4bff-414f-ae07-00584aef455a,94d10a24-04eb-4c18-bb42-ad6d7132efe2,2025-04-30 17:30:00
11176,baf7efb5-ae74-4501-8d98-692fe45f1724,3010c7a1-52b2-4df1-9922-8c67236ca4e3,b5e16dea-3fec-4449-a494-61ca2c85a0db,2025-04-30 17:30:00
11177,ec2c83f0-ff14-4231-ac4d-f45c4e401909,1b006fbf-6cea-4b4d-aebc-1c0868fa5c0b,681b0970-aac4-4e10-b71a-95904751de8a,2025-04-30 17:30:00
11178,401216ee-4f39-4ea8-9d49-1b41fb57f6c6,27d4f24d-232f-4cc3-9d32-93d40cf282d8,cb5b468c-5403-465e-8980-3105fc0a6570,2025-04-30 17:30:00


<hr>

## VaccineRecords Table

Set as empty database


In [23]:
vaccine_records_count = 0

# Vaccine Records Table
vaccine_records_data = {
    "id": [str(uuid.uuid4()) for _ in range(1, vaccine_records_count + 1)],
    "user_id": [random.choice(users_data["id"]) for _ in range(vaccine_records_count)],
    "booking_slot_id": [
        np.random.choice(booking_slots_data["id"], replace=False)
        for _ in range(vaccine_records_count)
    ],
    "status": [
        random.choice(["booked", "completed"]) for _ in range(vaccine_records_count)
    ],
}
vaccine_records_df = pd.DataFrame(vaccine_records_data)

display(vaccine_records_df)

,id,user_id,booking_slot_id,status


<hr>

## Insert Vaccination Data into SQLite Database

Delete the existing SQLite file before running the cell below to avoid appending to old SQLite database.


In [24]:
# Define SQLite database file path again
sqlite_db_path = "../data/vaccination_db.sqlite"

# Reconnect to SQLite database
conn = sqlite3.connect(sqlite_db_path)
cursor = conn.cursor()


# Function to insert DataFrame data into SQLite
def insert_csv_to_sqlite(
    csv_data: pd.DataFrame, table_name: str, conn: sqlite3.Connection
):
    csv_data.to_sql(table_name, conn, if_exists="append", index=False)


# Insert data into SQLite tables
insert_csv_to_sqlite(users_df, "Users", conn)
insert_csv_to_sqlite(clinics_df, "Clinics", conn)
insert_csv_to_sqlite(addresses_df, "Addresses", conn)
insert_csv_to_sqlite(vaccines_df, "Vaccines", conn)
insert_csv_to_sqlite(vaccine_criteria_df, "VaccineCriteria", conn)
insert_csv_to_sqlite(booking_slots_df, "BookingSlots", conn)
insert_csv_to_sqlite(vaccine_records_df, "VaccineRecords", conn)

# Commit changes and close connection
conn.commit()
conn.close()

<hr>

## Query SQLite Database


In [25]:
conn = sqlite3.connect("../data/vaccination_db.sqlite")
cursor = conn.cursor()

# Example: Fetch all users
cursor.execute("SELECT * FROM Users")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('812cf9f3-05be-429b-84d8-60b0fccac402', 'eb3c6dee-c95e-4595-92f5-9b574702369d', None, 'S9489143A', 'Mark', 'Johnson', 'mark.johnson@example.net', '1994-08-27', 'M', '$2b$12$kly/wBEGaHIDWw.1RWGNtOoz7I1UZbOqw7ShG8sUorskarWZij6lm', '2025-04-02 03:56:35', '2025-04-02 03:56:35')
('b14a41d8-5819-4685-93fa-d46434f32e0a', '6708a9a1-88fe-47f2-80a3-518a01d75fe9', '042e6bc3-6cd0-4808-909c-16fbde02d4ee', 'S6091645N', 'Stephanie', 'Miller', 'stephanie.miller@example.org', '1960-01-31', 'F', '$2b$12$Zpo8JyOjvPF2sHBZzakUO.WYt8wIsT9pp.2qXNLwjy10tVWVwHaaO', '2025-04-02 03:56:35', '2025-04-02 03:56:35')
('d9d303b7-240c-4320-ae6c-e3d7ef914ce3', '059d2e2a-8b8d-466c-a644-19ab8039288b', 'a0e4b2dc-4bff-414f-ae07-00584aef455a', 'S5962037V', 'Jonathan', 'Johnson', 'jonathan.johnson@example.com', '1959-04-03', 'M', '$2b$12$LGg25.ziP0jnuamBfeRx6.UvzRHcqLnyZjajdcBfcvqE5b8isBoCi', '2025-04-02 03:56:35', '2025-04-02 03:56:35')
('fcef2a1c-a9bc-4c27-a7bd-3aedc4ce4c53', '9c1728ef-581c-4956-a8ab-ee77eb775be8', '4acb36

In [26]:
db_filename = "../data/vaccination_db.sqlite"

try:
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = [row[0] for row in cursor.fetchall()]

    print("Tables in the database:")
    for table_name in table_names:
        print(table_name)

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

finally:
    if conn:
        conn.close()

Tables in the database:
Users
Clinics
Addresses
Vaccines
VaccineCriteria
BookingSlots
VaccineRecords
